# Use rdflib to do the queries

So import the necessary libraries:

In [2]:
from rdflib import Graph, Literal, Dataset, URIRef, Namespace
from rdflib.namespace import RDF, RDFS, Namespace
from rdflib.namespace import XSD
from rdflib import BNode
# import xml.etree.ElementTree as ET
# import re
# import html

# set up namespaces
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")
bflc = Namespace("http://id.loc.gov/ontologies/bflc/")
madsrdf = Namespace("http://www.loc.gov/mads/rdf/v1#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
lang = Namespace("http://id.loc.gov/vocabulary/iso639-2/")
locid = Namespace("http://id.loc.gov/vocabulary/identifiers/")
pxc = Namespace("https://w3id.org/zpid/ontology/classes/")
pxp = Namespace("https://w3id.org/zpid/ontology/properties/")
roles = Namespace("https://w3id.org/zpid/vocabs/roles/")
works = Namespace("https://w3id.org/zpid/resources/works/")
instances = Namespace("https://w3id.org/zpid/resources/instances/")

# set up graph and fill it with data from an rdf file:
g = Graph()
g.parse("ttl-data/bibframe_records.xml", format="xml")


<Graph identifier=N8d762ccc1a40494cb7a4921afaed6eb8 (<class 'rdflib.graph.Graph'>)>

# Show me all Records with a subtitle

In [3]:
origin_query = """
prefix bf: <http://id.loc.gov/ontologies/bibframe/> 
prefix bflc: <http://id.loc.gov/ontologies/bflc/> 
prefix lang: <http://id.loc.gov/vocabulary/iso639-2/> 
prefix locid: <http://id.loc.gov/vocabulary/identifiers/> 
prefix mads: <http://www.loc.gov/mads/rdf/v1#> 
prefix pxc: <https://w3id.org/zpid/ontology/classes/> 
prefix pxp: <https://w3id.org/zpid/ontology/properties/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix roles: <https://w3id.org/zpid/vocabs/roles/> 
prefix schema: <https://schema.org/> 
prefix works: <https://w3id.org/zpid/resources/works/> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT ?dfk ?subtitle
WHERE {
?instance bf:identifiedBy ?identifier. 
?identifier a pxc:DFK; rdf:value ?dfk . 
?instance bf:title/bf:subtitle ?subtitle .
} """

qres = g.query(origin_query)
for row in qres:
    # print(row)
    print(f"{row.dfk} hat Untertitel {row.subtitle}")

0390654 hat Untertitel Gespräche effektiv gestalten
0381631 hat Untertitel BDA-Befragung zur psychosozialen Unterstützung in der Akutmedizin im Herbst 2019
0390665 hat Untertitel Symptoms, risk factors, and treatment
0390735 hat Untertitel Eine Einführung
0390009 hat Untertitel Die vermittelnde Rolle von Akzeptanz, Machbarkeit und Kooperation
0390326 hat Untertitel Psychoanalyse, Sozialisationstheorie und Tiefenhermeneutik
0390043 hat Untertitel Beiträge zur 40. Jahrestagung des Arbeitskreises Klinische Psychologie in der Rehabilitation vom 12.11.-13.11.2021 online
0389961 hat Untertitel Longitudinale Betrachtung von Klassenklima, Lehrer-Schüler-Beziehung und sozialer Partizipation
0388779 hat Untertitel Somatic and depressive disorders, resilience and illness perception
0390680 hat Untertitel Ätiologie und Ansätze für die Therapie
0390316 hat Untertitel Jahresschwerpunkt 2022
0389999 hat Untertitel Ursachen, Folgen und Prävention wissenschaftlichen Fehlverhaltens
0390247 hat Untertite

# Check if there are records with no main title (TI was empty)

In [1]:
origin_query = """
prefix bf: <http://id.loc.gov/ontologies/bibframe/> 
prefix bflc: <http://id.loc.gov/ontologies/bflc/> 
prefix lang: <http://id.loc.gov/vocabulary/iso639-2/> 
prefix locid: <http://id.loc.gov/vocabulary/identifiers/> 
prefix mads: <http://www.loc.gov/mads/rdf/v1#> 
prefix pxc: <https://w3id.org/zpid/ontology/classes/> 
prefix pxp: <https://w3id.org/zpid/ontology/properties/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix roles: <https://w3id.org/zpid/vocabs/roles/> 
prefix schema: <https://schema.org/> 
prefix works: <https://w3id.org/zpid/resources/works/> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT ?dfk
WHERE {
?instance bf:identifiedBy ?identifier. 
?identifier a pxc:DFK; rdf:value ?dfk . 

filter not exists {?instance bf:title/bf:mainTitle ?maintitle .}
} """

qres = g.query(origin_query)
for row in qres:
    # print(row)
    print(f"{row.dfk} hat keinen Titel!")

NameError: name 'g' is not defined

# Check if there are Abstracts with no origin

In [5]:
# Two dont even have an abstract:
# (rdflib.term.Literal('0390504'),)
# (rdflib.term.Literal('0390502'),)

origin_query = """
SELECT DISTINCT ?dfk
WHERE {
?work bf:hasInstance ?instance .
?instance bf:identifiedBy ?identifier. 
?identifier a pxc:DFK; rdf:value ?dfk . 
# it must have an abstract to check (there are some without):
?work bf:summary ?abstract .

# now let's see if some abstracts have no origin string:
filter not exists {?abstract bf:adminMetadata/bflc:metadataLicensor ?abstract_origin .}

# or no editor:
#filter not exists {?abstract bf:adminMetadata/bf:descriptionModifier ?abstract_editor .}
}"""

qres = g.query(origin_query)
print(f"## Found {len(qres)} abstracts without origin string.")
for row in qres:
    # print(row)
    print(f"{row.dfk} has no origin string!")

## Found 0 abstracts without origin string.


# All dinstinct values of the abstract origin property

In [6]:
origin_query = """
SELECT DISTINCT ?abstract_origin
WHERE {
?work bf:hasInstance ?instance .
?instance bf:identifiedBy ?identifier. 
?identifier a pxc:DFK; rdf:value ?dfk . 
# it must have an abstract to check (there are some without):
?work bf:summary ?abstract .

# let's list all available origin strings:
?abstract bf:adminMetadata/bflc:metadataLicensor ?abstract_origin .

}"""

qres = g.query(origin_query)
print(f"## There are these {len(qres)} disctinct abstract origins:")
for count, row in enumerate(qres):
    # print(row)
    print(f"{count+1}. {row.abstract_origin}")

## There are these 4 disctinct abstract origins:
1. Original
2. DeepL
3. ZPID
4. FIS Bildung


# List affiliation countries without a country code (where non was found)
Because that means that my country id table is incomplete or the country name in STAR is faulty. 

In [16]:
country_query = """
prefix mads: <http://www.loc.gov/mads/rdf/v1#>
prefix bf: <http://id.loc.gov/ontologies/bibframe/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# i need a query that finds any mads:Country that has an rdfs:label, but no bf:identifiedBy:

SELECT ?dfk ?label
WHERE {
  ?work bf:contribution/mads:hasAffiliation/mads:hasAffiliationAddress/mads:country ?country .
  ?country a mads:Country ;
           rdfs:label ?label .
  ?work bf:hasInstance ?instance .
  ?instance bf:identifiedBy ?dfk_node .
  ?dfk_node a pxc:DFK; rdf:value ?dfk.
  FILTER NOT EXISTS { ?country bf:identifier ?id . ?id a locid:geonames. }
} 
"""

qres = g.query(country_query)
for row in qres:
    # print(row)
    print(f"{row.dfk}: {row.label} hat keine ID!")
